In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
"""
Adaptive Multi-Modal Federated Learning System 2025
===================================================

Intelligent system that:
1. Discovers available datasets automatically
2. Analyzes dataset characteristics and client resources
3. Makes adaptive decisions about training priority and resource allocation
4. Provides comprehensive results, visualizations, and adaptive strategies
5. Handles clients with 2, 3, or 4 modalities intelligently

Key Features:
- Dataset Discovery Engine
- Adaptive Training Scheduler  
- Resource-Aware Model Selection
- Comprehensive Results & Visualization
- Priority-Based Training Strategy
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split, Subset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import pandas as pd
import time
import psutil
import logging
import requests
import zipfile
import json
import pickle
from pathlib import Path
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Any, Optional, Union
from datetime import datetime
from PIL import Image
import re
import string
from collections import Counter, defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class DatasetCharacteristics:
    """Characteristics of a discovered dataset"""
    modality: str
    name: str
    size: int
    num_classes: int
    complexity_score: float  # 0-1, higher = more complex
    quality_score: float     # 0-1, higher = better quality
    estimated_training_time: float  # minutes
    memory_requirement: float       # GB
    priority_score: float = field(init=False)  # 0-1, higher = train first (calculated)
    
    def __post_init__(self):
        # Calculate priority based on multiple factors
        self.priority_score = (
            0.4 * self.quality_score +
            0.3 * (1.0 - self.complexity_score) +  # Easier datasets get higher priority
            0.2 * min(self.size / 10000, 1.0) +    # Larger datasets get some priority
            0.1 * (1.0 - self.memory_requirement / 8.0)  # Lower memory = higher priority
        )

@dataclass
class ClientResources:
    """Client computational resources"""
    available_memory_gb: float
    cpu_cores: int
    has_gpu: bool
    gpu_memory_gb: float
    estimated_time_budget_hours: float
    
    def get_resource_tier(self) -> str:
        """Classify client into resource tiers"""
        if self.has_gpu and self.gpu_memory_gb > 8 and self.available_memory_gb > 16:
            return "high"
        elif self.has_gpu and self.gpu_memory_gb > 4 and self.available_memory_gb > 8:
            return "medium"
        else:
            return "low"

@dataclass
class AdaptiveConfig:
    """Adaptive configuration that changes based on discoveries"""
    # Base configuration
    base_lr: float = 0.001
    base_batch_size: int = 32
    base_epochs: int = 10
    
    # Adaptive parameters (will be modified)
    current_lr: float = field(init=False)
    current_batch_size: int = field(init=False)
    current_epochs: int = field(init=False)
    
    # Discovery settings
    data_discovery_paths: List[str] = field(default_factory=lambda: [
        "/kaggle/working/datasets",
        "/kaggle/input",
        "./data",
        "./datasets"
    ])
    
    # Results and visualization
    save_detailed_results: bool = True
    generate_visualizations: bool = True
    save_adaptive_decisions: bool = True
    
    # Paths
    results_dir: str = "/kaggle/working/adaptive_results"
    plots_dir: str = "/kaggle/working/plots"
    models_dir: str = "/kaggle/working/models"
    logs_dir: str = "/kaggle/working/logs"
    
    def __post_init__(self):
        # Initialize adaptive parameters
        self.current_lr = self.base_lr
        self.current_batch_size = self.base_batch_size
        self.current_epochs = self.base_epochs
        
        # Create directories
        for directory in [self.results_dir, self.plots_dir, self.models_dir, self.logs_dir]:
            Path(directory).mkdir(parents=True, exist_ok=True)

class DatasetDiscoveryEngine:
    """Automatically discovers and analyzes available datasets"""
    
    def __init__(self, config: AdaptiveConfig):
        self.config = config
        self.discovered_datasets = {}
        self.dataset_characteristics = {}
        
    def discover_datasets(self) -> Dict[str, DatasetCharacteristics]:
        """Discover available datasets and analyze their characteristics"""
        logger.info("🔍 Starting dataset discovery...")
        
        # For demo purposes, simulate different client scenarios
        # In real implementation, this would scan actual directories
        scenarios = self._generate_client_scenarios()
        
        # Randomly select a scenario (simulate different clients)
        scenario_name = np.random.choice(list(scenarios.keys()))
        selected_scenario = scenarios[scenario_name]
        
        logger.info(f"📊 Simulating client scenario: '{scenario_name}'")
        logger.info(f"🗂️  Available modalities: {list(selected_scenario.keys())}")
        
        # Analyze each discovered dataset
        for modality, dataset_info in selected_scenario.items():
            characteristics = self._analyze_dataset_characteristics(modality, dataset_info)
            self.dataset_characteristics[modality] = characteristics
            
            logger.info(f"  📈 {modality}: Quality={characteristics.quality_score:.2f}, "
                       f"Complexity={characteristics.complexity_score:.2f}, "
                       f"Priority={characteristics.priority_score:.2f}")
        
        # Save discovery results
        self._save_discovery_results()
        
        return self.dataset_characteristics
    
    def _generate_client_scenarios(self) -> Dict[str, Dict]:
        """Generate different client scenarios with varying dataset combinations"""
        scenarios = {
            "medical_center": {
                "vision": {"type": "medical_images", "size": 8000, "quality": 0.9},
                "sensor": {"type": "patient_monitoring", "size": 12000, "quality": 0.85},
                "text": {"type": "clinical_notes", "size": 5000, "quality": 0.7}
            },
            "smart_city": {
                "vision": {"type": "traffic_cameras", "size": 15000, "quality": 0.8},
                "sensor": {"type": "iot_sensors", "size": 20000, "quality": 0.9},
                "multimodal": {"type": "surveillance", "size": 3000, "quality": 0.7}
            },
            "mobile_devices": {
                "sensor": {"type": "smartphone_sensors", "size": 25000, "quality": 0.95},
                "text": {"type": "user_messages", "size": 18000, "quality": 0.6},
                "multimodal": {"type": "social_media", "size": 8000, "quality": 0.75}
            },
            "research_lab": {
                "vision": {"type": "scientific_images", "size": 12000, "quality": 0.95},
                "text": {"type": "research_papers", "size": 10000, "quality": 0.9},
                "sensor": {"type": "lab_equipment", "size": 8000, "quality": 0.8},
                "multimodal": {"type": "experiments", "size": 4000, "quality": 0.85}
            },
            "edge_device": {
                "sensor": {"type": "environmental", "size": 6000, "quality": 0.7},
                "vision": {"type": "security_camera", "size": 4000, "quality": 0.6}
            }
        }
        return scenarios
    
    def _analyze_dataset_characteristics(self, modality: str, dataset_info: Dict) -> DatasetCharacteristics:
        """Analyze characteristics of a discovered dataset"""
        
        # Extract basic info
        dataset_type = dataset_info["type"]
        size = dataset_info["size"]
        quality = dataset_info["quality"]
        
        # Determine complexity based on dataset type and modality
        complexity_map = {
            "vision": {
                "medical_images": 0.9, "traffic_cameras": 0.6, "scientific_images": 0.95,
                "security_camera": 0.4
            },
            "text": {
                "clinical_notes": 0.8, "user_messages": 0.3, "research_papers": 0.9
            },
            "sensor": {
                "patient_monitoring": 0.7, "iot_sensors": 0.5, "smartphone_sensors": 0.6,
                "lab_equipment": 0.8, "environmental": 0.4
            },
            "multimodal": {
                "surveillance": 0.85, "social_media": 0.7, "experiments": 0.9
            }
        }
        
        complexity = complexity_map.get(modality, {}).get(dataset_type, 0.5)
        
        # Estimate training requirements
        base_time_per_sample = {"vision": 0.01, "text": 0.005, "sensor": 0.003, "multimodal": 0.02}
        estimated_time = (size * base_time_per_sample.get(modality, 0.01)) / 60  # minutes
        
        memory_map = {"vision": 2.0, "text": 1.0, "sensor": 0.5, "multimodal": 3.0}
        memory_req = memory_map.get(modality, 1.0) * (size / 10000)
        
        # Determine number of classes (simplified)
        num_classes_map = {
            "vision": {"medical_images": 5, "traffic_cameras": 10, "scientific_images": 20, "security_camera": 3},
            "text": {"clinical_notes": 8, "user_messages": 3, "research_papers": 15},
            "sensor": {"patient_monitoring": 6, "iot_sensors": 4, "smartphone_sensors": 6, "lab_equipment": 8, "environmental": 4},
            "multimodal": {"surveillance": 2, "social_media": 4, "experiments": 10}
        }
        
        num_classes = num_classes_map.get(modality, {}).get(dataset_type, 5)
        
        return DatasetCharacteristics(
            modality=modality,
            name=dataset_type,
            size=size,
            num_classes=num_classes,
            complexity_score=complexity,
            quality_score=quality,
            estimated_training_time=estimated_time,
            memory_requirement=memory_req
        )
    
    def _save_discovery_results(self):
        """Save dataset discovery results"""
        discovery_results = {
            "timestamp": datetime.now().isoformat(),
            "discovered_datasets": {
                modality: {
                    "name": char.name,
                    "size": char.size,
                    "quality_score": char.quality_score,
                    "complexity_score": char.complexity_score,
                    "priority_score": char.priority_score,
                    "estimated_training_time": char.estimated_training_time,
                    "memory_requirement": char.memory_requirement
                }
                for modality, char in self.dataset_characteristics.items()
            }
        }
        
        # Save as JSON
        with open(f"{self.config.results_dir}/dataset_discovery.json", "w") as f:
            json.dump(discovery_results, f, indent=2)
        
        # Save as CSV for easy analysis
        df_data = []
        for modality, char in self.dataset_characteristics.items():
            df_data.append({
                "modality": modality,
                "name": char.name,
                "size": char.size,
                "quality": char.quality_score,
                "complexity": char.complexity_score,
                "priority": char.priority_score,
                "est_time_min": char.estimated_training_time,
                "memory_gb": char.memory_requirement
            })
        
        df = pd.DataFrame(df_data)
        df.to_csv(f"{self.config.results_dir}/dataset_analysis.csv", index=False)
        
        logger.info(f"💾 Discovery results saved to {self.config.results_dir}")

class AdaptiveTrainingScheduler:
    """Intelligent scheduler that adapts training based on resources and priorities"""
    
    def __init__(self, config: AdaptiveConfig, client_resources: ClientResources):
        self.config = config
        self.client_resources = client_resources
        self.training_plan = []
        self.adaptive_decisions = []
        
    def create_adaptive_training_plan(self, dataset_characteristics: Dict[str, DatasetCharacteristics]) -> List[Dict]:
        """Create an adaptive training plan based on resources and priorities"""
        logger.info("🧠 Creating adaptive training plan...")
        
        # Sort datasets by priority
        sorted_datasets = sorted(
            dataset_characteristics.items(),
            key=lambda x: x[1].priority_score,
            reverse=True
        )
        
        logger.info("📋 Dataset priority order:")
        for i, (modality, char) in enumerate(sorted_datasets):
            logger.info(f"  {i+1}. {modality} (priority: {char.priority_score:.3f})")
        
        # Calculate resource allocation
        total_time_budget = self.client_resources.estimated_time_budget_hours * 60  # minutes
        total_memory_budget = self.client_resources.available_memory_gb
        
        # Adaptive resource allocation
        training_plan = []
        remaining_time = total_time_budget
        remaining_memory = total_memory_budget
        
        for modality, characteristics in sorted_datasets:
            # Check if we can fit this dataset
            if (characteristics.estimated_training_time <= remaining_time * 1.2 and  # 20% buffer
                characteristics.memory_requirement <= remaining_memory):
                
                # Adapt training parameters based on resources and dataset
                adapted_params = self._adapt_training_parameters(characteristics)
                
                training_plan.append({
                    "modality": modality,
                    "characteristics": characteristics,
                    "adapted_params": adapted_params,
                    "estimated_time": characteristics.estimated_training_time,
                    "allocated_time": min(characteristics.estimated_training_time * 1.5, remaining_time * 0.8)
                })
                
                remaining_time -= characteristics.estimated_training_time
                remaining_memory -= characteristics.memory_requirement * 0.5  # Assume 50% memory reuse
                
                decision = f"✅ Including {modality}: Time={characteristics.estimated_training_time:.1f}min, Memory={characteristics.memory_requirement:.1f}GB"
                logger.info(f"  {decision}")
                self.adaptive_decisions.append(decision)
            else:
                decision = f"❌ Skipping {modality}: Insufficient resources (Time: {characteristics.estimated_training_time:.1f}/{remaining_time:.1f}min, Memory: {characteristics.memory_requirement:.1f}/{remaining_memory:.1f}GB)"
                logger.info(f"  {decision}")
                self.adaptive_decisions.append(decision)
        
        self.training_plan = training_plan
        self._save_training_plan()
        
        return training_plan
    
    def _adapt_training_parameters(self, characteristics: DatasetCharacteristics) -> Dict:
        """Adapt training parameters based on dataset characteristics and resources"""
        
        # Base parameters
        adapted_params = {
            "epochs": self.config.base_epochs,
            "batch_size": self.config.base_batch_size,
            "learning_rate": self.config.base_lr,
            "model_complexity": "medium"
        }
        
        # Adapt based on resource tier
        resource_tier = self.client_resources.get_resource_tier()
        
        if resource_tier == "high":
            adapted_params["batch_size"] = min(128, self.config.base_batch_size * 4)
            adapted_params["model_complexity"] = "high"
        elif resource_tier == "low":
            adapted_params["batch_size"] = max(8, self.config.base_batch_size // 2)
            adapted_params["epochs"] = max(5, self.config.base_epochs // 2)
            adapted_params["model_complexity"] = "low"
        
        # Adapt based on dataset characteristics
        if characteristics.complexity_score > 0.8:
            adapted_params["epochs"] = int(adapted_params["epochs"] * 1.5)
            adapted_params["learning_rate"] *= 0.5  # Lower LR for complex datasets
        
        if characteristics.quality_score < 0.6:
            adapted_params["epochs"] = max(5, int(adapted_params["epochs"] * 0.7))  # Less training for low quality
        
        # Memory constraints
        if characteristics.memory_requirement > self.client_resources.available_memory_gb * 0.5:
            adapted_params["batch_size"] = max(8, adapted_params["batch_size"] // 2)
            adapted_params["model_complexity"] = "low"
        
        return adapted_params
    
    def _save_training_plan(self):
        """Save the adaptive training plan"""
        plan_data = {
            "timestamp": datetime.now().isoformat(),
            "client_resources": {
                "memory_gb": self.client_resources.available_memory_gb,
                "cpu_cores": self.client_resources.cpu_cores,
                "has_gpu": self.client_resources.has_gpu,
                "gpu_memory_gb": self.client_resources.gpu_memory_gb,
                "time_budget_hours": self.client_resources.estimated_time_budget_hours,
                "resource_tier": self.client_resources.get_resource_tier()
            },
            "training_plan": [
                {
                    "modality": item["modality"],
                    "priority_score": item["characteristics"].priority_score,
                    "dataset_size": item["characteristics"].size,
                    "adapted_epochs": item["adapted_params"]["epochs"],
                    "adapted_batch_size": item["adapted_params"]["batch_size"],
                    "adapted_lr": item["adapted_params"]["learning_rate"],
                    "model_complexity": item["adapted_params"]["model_complexity"],
                    "estimated_time_min": item["estimated_time"],
                    "allocated_time_min": item["allocated_time"]
                }
                for item in self.training_plan
            ],
            "adaptive_decisions": self.adaptive_decisions
        }
        
        with open(f"{self.config.results_dir}/adaptive_training_plan.json", "w") as f:
            json.dump(plan_data, f, indent=2)
        
        logger.info(f"📋 Training plan saved with {len(self.training_plan)} modalities")

class ComprehensiveResultsManager:
    """Manages comprehensive results, metrics, and visualizations"""
    
    def __init__(self, config: AdaptiveConfig):
        self.config = config
        self.training_results = {}
        self.training_history = defaultdict(dict)
        self.resource_usage = []
        self.adaptive_decisions = []
        
    def log_training_start(self, modality: str, adapted_params: Dict):
        """Log the start of training for a modality"""
        start_info = {
            "modality": modality,
            "start_time": datetime.now().isoformat(),
            "adapted_params": adapted_params,
            "start_memory_gb": psutil.virtual_memory().available / (1024**3),
            "start_cpu_percent": psutil.cpu_percent(interval=1)
        }
        
        if torch.cuda.is_available():
            start_info["start_gpu_memory_gb"] = torch.cuda.memory_allocated() / (1024**3)
        
        self.training_results[modality] = start_info
        logger.info(f"📊 Started training {modality} with {adapted_params}")
    
    def log_epoch_results(self, modality: str, epoch: int, train_acc: float, val_acc: float, 
                         train_loss: float, val_loss: float, lr: float):
        """Log results for each epoch"""
        if modality not in self.training_history:
            self.training_history[modality] = {
                "epochs": [], "train_acc": [], "val_acc": [],
                "train_loss": [], "val_loss": [], "learning_rates": []
            }
        
        self.training_history[modality]["epochs"].append(epoch)
        self.training_history[modality]["train_acc"].append(train_acc)
        self.training_history[modality]["val_acc"].append(val_acc)
        self.training_history[modality]["train_loss"].append(train_loss)
        self.training_history[modality]["val_loss"].append(val_loss)
        self.training_history[modality]["learning_rates"].append(lr)
    
    def log_training_end(self, modality: str, final_results: Dict):
        """Log the end of training for a modality"""
        end_info = {
            "end_time": datetime.now().isoformat(),
            "final_train_accuracy": final_results.get("final_train_accuracy", 0),
            "best_val_accuracy": final_results.get("best_val_accuracy", 0),
            "epochs_completed": final_results.get("epochs_trained", 0),
            "end_memory_gb": psutil.virtual_memory().available / (1024**3),
            "end_cpu_percent": psutil.cpu_percent(interval=1)
        }
        
        if torch.cuda.is_available():
            end_info["end_gpu_memory_gb"] = torch.cuda.memory_allocated() / (1024**3)
        
        # Calculate training duration
        start_time = datetime.fromisoformat(self.training_results[modality]["start_time"])
        end_time = datetime.fromisoformat(end_info["end_time"])
        duration_minutes = (end_time - start_time).total_seconds() / 60
        end_info["training_duration_minutes"] = duration_minutes
        
        self.training_results[modality].update(end_info)
        
        logger.info(f"✅ Completed {modality}: {end_info['best_val_accuracy']:.2f}% accuracy in {duration_minutes:.1f} minutes")
    
    def save_comprehensive_results(self):
        """Save all results and generate visualizations"""
        logger.info("💾 Saving comprehensive results and generating visualizations...")
        
        # Save detailed results
        self._save_detailed_metrics()
        
        # Generate visualizations
        if self.config.generate_visualizations:
            self._generate_training_curves()
            self._generate_performance_comparison()
            self._generate_resource_usage_plots()
            self._generate_adaptive_decisions_summary()
        
        # Save training history
        self._save_training_history()
        
        # Generate final report
        self._generate_final_report()
        
        logger.info(f"📊 All results saved to {self.config.results_dir}")
        logger.info(f"📈 Visualizations saved to {self.config.plots_dir}")
    
    def _save_detailed_metrics(self):
        """Save detailed metrics for each modality"""
        detailed_results = {
            "experiment_info": {
                "timestamp": datetime.now().isoformat(),
                "total_modalities_trained": len(self.training_results),
                "total_experiment_time_minutes": sum(
                    result.get("training_duration_minutes", 0) 
                    for result in self.training_results.values()
                )
            },
            "modality_results": self.training_results,
            "training_history": dict(self.training_history)
        }
        
        # Save as JSON
        with open(f"{self.config.results_dir}/detailed_results.json", "w") as f:
            json.dump(detailed_results, f, indent=2, default=str)
        
        # Save as CSV for easy analysis
        csv_data = []
        for modality, results in self.training_results.items():
            csv_data.append({
                "modality": modality,
                "best_val_accuracy": results.get("best_val_accuracy", 0),
                "final_train_accuracy": results.get("final_train_accuracy", 0),
                "epochs_completed": results.get("epochs_completed", 0),
                "training_duration_min": results.get("training_duration_minutes", 0),
                "adapted_batch_size": results.get("adapted_params", {}).get("batch_size", 0),
                "adapted_lr": results.get("adapted_params", {}).get("learning_rate", 0),
                "model_complexity": results.get("adapted_params", {}).get("model_complexity", ""),
                "memory_used_gb": results.get("start_memory_gb", 0) - results.get("end_memory_gb", 0)
            })
        
        df = pd.DataFrame(csv_data)
        df.to_csv(f"{self.config.results_dir}/results_summary.csv", index=False)
    
    def _generate_training_curves(self):
        """Generate training curve plots for each modality"""
        if not self.training_history:
            return
            
        n_modalities = len(self.training_history)
        fig, axes = plt.subplots(2, n_modalities, figsize=(5*n_modalities, 10))
        
        if n_modalities == 1:
            axes = axes.reshape(2, 1)
        
        for idx, (modality, history) in enumerate(self.training_history.items()):
            # Accuracy plot
            axes[0, idx].plot(history["epochs"], history["train_acc"], 'b-', label='Train Accuracy', linewidth=2)
            axes[0, idx].plot(history["epochs"], history["val_acc"], 'r-', label='Val Accuracy', linewidth=2)
            axes[0, idx].set_title(f'{modality.title()} - Accuracy')
            axes[0, idx].set_xlabel('Epoch')
            axes[0, idx].set_ylabel('Accuracy (%)')
            axes[0, idx].legend()
            axes[0, idx].grid(True, alpha=0.3)
            
            # Loss plot
            axes[1, idx].plot(history["epochs"], history["train_loss"], 'b-', label='Train Loss', linewidth=2)
            axes[1, idx].plot(history["epochs"], history["val_loss"], 'r-', label='Val Loss', linewidth=2)
            axes[1, idx].set_title(f'{modality.title()} - Loss')
            axes[1, idx].set_xlabel('Epoch')
            axes[1, idx].set_ylabel('Loss')
            axes[1, idx].legend()
            axes[1, idx].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f"{self.config.plots_dir}/training_curves.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    def _generate_performance_comparison(self):
        """Generate performance comparison charts"""
        if not self.training_results:
            return
            
        modalities = list(self.training_results.keys())
        accuracies = [self.training_results[mod].get("best_val_accuracy", 0) for mod in modalities]
        training_times = [self.training_results[mod].get("training_duration_minutes", 0) for mod in modalities]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Accuracy comparison
        bars1 = ax1.bar(modalities, accuracies, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
        ax1.set_title('Best Validation Accuracy by Modality', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Accuracy (%)')
        ax1.set_ylim(0, 100)
        
        # Add value labels on bars
        for bar, acc in zip(bars1, accuracies):
            height = bar.get_height()
            ax1.annotate(f'{acc:.1f}%', xy=(bar.get_x() + bar.get_width()/2, height),
                        xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
        
        # Training time comparison  
        bars2 = ax2.bar(modalities, training_times, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
        ax2.set_title('Training Duration by Modality', fontsize=14, fontweight='bold')
        ax2.set_ylabel('Training Time (minutes)')
        
        # Add value labels on bars
        for bar, time in zip(bars2, training_times):
            height = bar.get_height()
            ax2.annotate(f'{time:.1f}m', xy=(bar.get_x() + bar.get_width()/2, height),
                        xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
        
        plt.tight_layout()
        plt.savefig(f"{self.config.plots_dir}/performance_comparison.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    def _generate_resource_usage_plots(self):
        """Generate resource usage visualization"""
        # Create a resource efficiency plot
        if not self.training_results:
            return
            
        modalities = list(self.training_results.keys())
        accuracies = [self.training_results[mod].get("best_val_accuracy", 0) for mod in modalities]
        times = [self.training_results[mod].get("training_duration_minutes", 1) for mod in modalities]
        
        # Calculate efficiency (accuracy per minute)
        efficiency = [acc/time for acc, time in zip(accuracies, times)]
        
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        
        scatter = ax.scatter(times, accuracies, s=[e*10 for e in efficiency], 
                            c=efficiency, cmap='viridis', alpha=0.7)
        
        # Add labels for each point
        for i, modality in enumerate(modalities):
            ax.annotate(modality, (times[i], accuracies[i]), 
                       xytext=(5, 5), textcoords='offset points')
        
        ax.set_xlabel('Training Time (minutes)')
        ax.set_ylabel('Best Validation Accuracy (%)')
        ax.set_title('Training Efficiency: Accuracy vs Time\n(Bubble size = Efficiency)')
        
        # Add colorbar
        cbar = plt.colorbar(scatter)
        cbar.set_label('Efficiency (Accuracy/Minute)')
        
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f"{self.config.plots_dir}/resource_efficiency.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    def _generate_adaptive_decisions_summary(self):
        """Generate summary of adaptive decisions made"""
        # Create a summary visualization of adaptive decisions
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Collect adaptive parameter information
        modalities = []
        complexities = []
        batch_sizes = []
        learning_rates = []
        epochs = []
        
        for modality, results in self.training_results.items():
            adapted_params = results.get("adapted_params", {})
            modalities.append(modality)
            complexities.append(adapted_params.get("model_complexity", "medium"))
            batch_sizes.append(adapted_params.get("batch_size", 32))
            learning_rates.append(adapted_params.get("learning_rate", 0.001))
            epochs.append(adapted_params.get("epochs", 10))
        
        # Create a table showing adaptive decisions
        table_data = []
        for i, mod in enumerate(modalities):
            table_data.append([
                mod.title(),
                complexities[i],
                f"{batch_sizes[i]}",
                f"{learning_rates[i]:.4f}",
                f"{epochs[i]}",
                f"{self.training_results[mod].get('best_val_accuracy', 0):.1f}%"
            ])
        
        # Create table
        ax.axis('tight')
        ax.axis('off')
        
        table = ax.table(cellText=table_data,
                        colLabels=['Modality', 'Model Complexity', 'Batch Size', 'Learning Rate', 'Epochs', 'Best Accuracy'],
                        cellLoc='center',
                        loc='center')
        
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.5)
        
        # Style the table
        for i in range(len(table_data) + 1):
            for j in range(6):
                if i == 0:  # Header row
                    table[(i, j)].set_facecolor('#4ECDC4')
                    table[(i, j)].set_text_props(weight='bold')
                else:
                    table[(i, j)].set_facecolor('#F8F9FA' if i % 2 == 0 else 'white')
        
        ax.set_title('Adaptive Training Decisions Summary', fontsize=16, fontweight='bold', pad=20)
        
        plt.tight_layout()
        plt.savefig(f"{self.config.plots_dir}/adaptive_decisions.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    def _save_training_history(self):
        """Save training history in multiple formats"""
        # Save as pickle for easy loading in Python
        with open(f"{self.config.results_dir}/training_history.pkl", "wb") as f:
            pickle.dump(dict(self.training_history), f)
        
        # Save as CSV for each modality
        for modality, history in self.training_history.items():
            df = pd.DataFrame(history)
            df.to_csv(f"{self.config.results_dir}/history_{modality}.csv", index=False)
    
    def _generate_final_report(self):
        """Generate a comprehensive final report"""
        report = []
        report.append("# Adaptive Multi-Modal Federated Learning - Final Report")
        report.append(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append("")
        
        # Executive Summary
        report.append("## Executive Summary")
        total_modalities = len(self.training_results)
        avg_accuracy = np.mean([r.get("best_val_accuracy", 0) for r in self.training_results.values()])
        total_time = sum([r.get("training_duration_minutes", 0) for r in self.training_results.values()])
        
        report.append(f"- **Total Modalities Trained**: {total_modalities}")
        report.append(f"- **Average Best Accuracy**: {avg_accuracy:.2f}%")
        report.append(f"- **Total Training Time**: {total_time:.1f} minutes")
        report.append("")
        
        # Individual Results
        report.append("## Individual Modality Results")
        for modality, results in self.training_results.items():
            report.append(f"### {modality.title()}")
            report.append(f"- **Best Validation Accuracy**: {results.get('best_val_accuracy', 0):.2f}%")
            report.append(f"- **Training Duration**: {results.get('training_duration_minutes', 0):.1f} minutes")
            report.append(f"- **Epochs Completed**: {results.get('epochs_completed', 0)}")
            
            adapted_params = results.get('adapted_params', {})
            report.append(f"- **Adapted Parameters**:")
            report.append(f"  - Model Complexity: {adapted_params.get('model_complexity', 'N/A')}")
            report.append(f"  - Batch Size: {adapted_params.get('batch_size', 'N/A')}")
            report.append(f"  - Learning Rate: {adapted_params.get('learning_rate', 'N/A')}")
            report.append("")
        
        # Adaptive Insights
        report.append("## Adaptive System Insights")
        if total_modalities > 1:
            best_modality = max(self.training_results.items(), key=lambda x: x[1].get("best_val_accuracy", 0))
            fastest_modality = min(self.training_results.items(), key=lambda x: x[1].get("training_duration_minutes", float('inf')))
            
            report.append(f"- **Best Performing Modality**: {best_modality[0]} ({best_modality[1].get('best_val_accuracy', 0):.2f}%)")
            report.append(f"- **Fastest Training Modality**: {fastest_modality[0]} ({fastest_modality[1].get('training_duration_minutes', 0):.1f} minutes)")
        
        report.append("")
        report.append("## Files Generated")
        report.append("- `detailed_results.json`: Complete results in JSON format")
        report.append("- `results_summary.csv`: Summary results in CSV format")
        report.append("- `training_curves.png`: Training accuracy and loss curves")
        report.append("- `performance_comparison.png`: Performance comparison charts")
        report.append("- `resource_efficiency.png`: Resource efficiency analysis")
        report.append("- `adaptive_decisions.png`: Summary of adaptive decisions")
        
        # Save report
        with open(f"{self.config.results_dir}/final_report.md", "w") as f:
            f.write("\n".join(report))

# Simple dataset classes for demo (using the same ones from before but simplified)
class SimpleVisionDataset(Dataset):
    def __init__(self, size: int, num_classes: int):
        self.data = [(torch.randn(3, 224, 224), i % num_classes) for i in range(size)]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data, label = self.data[idx]
        return {'data': data, 'label': label, 'modality': 'vision'}

class SimpleTextDataset(Dataset):
    def __init__(self, size: int, num_classes: int):
        self.data = [(torch.randint(0, 1000, (50,)), i % num_classes) for i in range(size)]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data, label = self.data[idx]
        return {'data': data, 'label': label, 'modality': 'text'}

class SimpleSensorDataset(Dataset):
    def __init__(self, size: int, num_classes: int):
        self.data = [(torch.randn(6, 128), i % num_classes) for i in range(size)]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data, label = self.data[idx]
        return {'data': data, 'label': label, 'modality': 'sensor'}

class SimpleMultiModalDataset(Dataset):
    def __init__(self, size: int, num_classes: int):
        self.data = [({
            'image': torch.randn(3, 224, 224),
            'text': torch.randint(0, 1000, (20,))
        }, i % num_classes) for i in range(size)]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data, label = self.data[idx]
        return {'data': data, 'label': label, 'modality': 'multimodal'}

# Simple model classes
class SimpleVisionModel(nn.Module):
    def __init__(self, num_classes: int, complexity: str = "medium"):
        super().__init__()
        if complexity == "low":
            self.features = nn.Sequential(
                nn.Conv2d(3, 32, 7, stride=4), nn.ReLU(),
                nn.AdaptiveAvgPool2d(4), nn.Flatten(),
                nn.Linear(32*16, num_classes)
            )
        else:
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, 7, stride=2), nn.ReLU(),
                nn.Conv2d(64, 128, 5, stride=2), nn.ReLU(),
                nn.AdaptiveAvgPool2d(4), nn.Flatten(),
                nn.Linear(128*16, 256), nn.ReLU(),
                nn.Linear(256, num_classes)
            )
    
    def forward(self, x):
        return self.features(x)

class SimpleTextModel(nn.Module):
    def __init__(self, vocab_size: int, num_classes: int, complexity: str = "medium"):
        super().__init__()
        embed_dim = 64 if complexity == "low" else 128
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, embed_dim, batch_first=True)
        self.classifier = nn.Linear(embed_dim, num_classes)
    
    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        return self.classifier(hidden[-1])

class SimpleSensorModel(nn.Module):
    def __init__(self, num_classes: int, complexity: str = "medium"):
        super().__init__()
        channels = 32 if complexity == "low" else 64
        self.features = nn.Sequential(
            nn.Conv1d(6, channels, 7), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1), nn.Flatten(),
            nn.Linear(channels, num_classes)
        )
    
    def forward(self, x):
        return self.features(x)

class SimpleMultiModalModel(nn.Module):
    def __init__(self, vocab_size: int, num_classes: int, complexity: str = "medium"):
        super().__init__()
        embed_dim = 64 if complexity == "low" else 128
        
        self.vision_encoder = nn.Sequential(
            nn.Conv2d(3, 32, 7, stride=4), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1), nn.Flatten(),
            nn.Linear(32, embed_dim)
        )
        
        self.text_encoder = nn.Sequential(
            nn.Embedding(vocab_size, embed_dim),
            nn.LSTM(embed_dim, embed_dim, batch_first=True)
        )
        
        self.classifier = nn.Linear(embed_dim * 2, num_classes)
    
    def forward(self, x):
        image_features = self.vision_encoder(x['image'])
        
        text_embedded = self.text_encoder[0](x['text'])
        _, (text_hidden, _) = self.text_encoder[1](text_embedded)
        text_features = text_hidden[-1]
        
        fused = torch.cat([image_features, text_features], dim=1)
        return self.classifier(fused)

# Main Adaptive System
class AdaptiveMultiModalFL:
    """Main adaptive multi-modal federated learning system"""
    
    def __init__(self, time_budget_hours: float = 2.0):
        # Initialize configuration
        self.config = AdaptiveConfig()
        
        # Simulate client resources (in real system, would detect automatically)
        memory = psutil.virtual_memory()
        self.client_resources = ClientResources(
            available_memory_gb=memory.available / (1024**3),
            cpu_cores=psutil.cpu_count(),
            has_gpu=torch.cuda.is_available(),
            gpu_memory_gb=torch.cuda.get_device_properties(0).total_memory / (1024**3) if torch.cuda.is_available() else 0,
            estimated_time_budget_hours=time_budget_hours
        )
        
        # Initialize components
        self.discovery_engine = DatasetDiscoveryEngine(self.config)
        self.scheduler = AdaptiveTrainingScheduler(self.config, self.client_resources)
        self.results_manager = ComprehensiveResultsManager(self.config)
        
        # Data and models
        self.datasets = {}
        self.models = {}
        
        logger.info("🚀 Initialized Adaptive Multi-Modal FL System")
        logger.info(f"💻 Client Resources: {self.client_resources.get_resource_tier()} tier")
        logger.info(f"⏰ Time Budget: {time_budget_hours} hours")
    
    def run_adaptive_experiment(self):
        """Run the complete adaptive experiment"""
        logger.info("🔬 Starting Adaptive Multi-Modal FL Experiment...")
        
        # Step 1: Discover available datasets
        dataset_characteristics = self.discovery_engine.discover_datasets()
        
        # Step 2: Create adaptive training plan
        training_plan = self.scheduler.create_adaptive_training_plan(dataset_characteristics)
        
        if not training_plan:
            logger.error("❌ No datasets can be trained with current resources!")
            return None
        
        # Step 3: Load datasets and initialize models
        self._load_datasets_and_models(training_plan)
        
        # Step 4: Execute adaptive training
        self._execute_adaptive_training(training_plan)
        
        # Step 5: Save comprehensive results
        self.results_manager.save_comprehensive_results()
        
        logger.info("🎉 Adaptive experiment completed successfully!")
        return self.results_manager.training_results
    
    def _load_datasets_and_models(self, training_plan: List[Dict]):
        """Load datasets and initialize models based on training plan"""
        logger.info("📚 Loading datasets and initializing models...")
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for plan_item in training_plan:
            modality = plan_item["modality"]
            characteristics = plan_item["characteristics"]
            adapted_params = plan_item["adapted_params"]
            
            # Create dataset (simplified for demo)
            if modality == "vision":
                dataset = SimpleVisionDataset(characteristics.size, characteristics.num_classes)
                model = SimpleVisionModel(characteristics.num_classes, adapted_params["model_complexity"])
            elif modality == "text":
                dataset = SimpleTextDataset(characteristics.size, characteristics.num_classes)
                model = SimpleTextModel(1000, characteristics.num_classes, adapted_params["model_complexity"])
            elif modality == "sensor":
                dataset = SimpleSensorDataset(characteristics.size, characteristics.num_classes)
                model = SimpleSensorModel(characteristics.num_classes, adapted_params["model_complexity"])
            elif modality == "multimodal":
                dataset = SimpleMultiModalDataset(characteristics.size, characteristics.num_classes)
                model = SimpleMultiModalModel(1000, characteristics.num_classes, adapted_params["model_complexity"])
            
            # Split dataset
            train_size = int(0.8 * len(dataset))
            val_size = len(dataset) - train_size
            train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
            
            # Create dataloaders
            train_loader = DataLoader(
                train_dataset,
                batch_size=adapted_params["batch_size"],
                shuffle=True
            )
            val_loader = DataLoader(
                val_dataset,
                batch_size=adapted_params["batch_size"],
                shuffle=False
            )
            
            self.datasets[modality] = {"train": train_loader, "val": val_loader}
            self.models[modality] = model.to(device)
            
            logger.info(f"  ✅ {modality}: {len(train_dataset)} train, {len(val_dataset)} val samples")
    
    def _execute_adaptive_training(self, training_plan: List[Dict]):
        """Execute training according to the adaptive plan"""
        logger.info("🎯 Executing adaptive training...")
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for plan_item in training_plan:
            modality = plan_item["modality"]
            adapted_params = plan_item["adapted_params"]
            
            logger.info(f"\n🔥 Training {modality} with adaptive parameters...")
            
            # Log training start
            self.results_manager.log_training_start(modality, adapted_params)
            
            # Get model and data
            model = self.models[modality]
            train_loader = self.datasets[modality]["train"]
            val_loader = self.datasets[modality]["val"]
            
            # Setup training
            optimizer = torch.optim.AdamW(
                model.parameters(),
                lr=adapted_params["learning_rate"],
                weight_decay=1e-4
            )
            criterion = nn.CrossEntropyLoss()
            
            best_val_acc = 0.0
            
            # Training loop
            for epoch in range(adapted_params["epochs"]):
                # Training phase
                model.train()
                train_loss = 0.0
                train_correct = 0
                train_total = 0
                
                for batch in train_loader:
                    if isinstance(batch['data'], dict):
                        data = {k: v.to(device) for k, v in batch['data'].items()}
                    else:
                        data = batch['data'].to(device)
                    
                    labels = batch['label'].to(device)
                    
                    optimizer.zero_grad()
                    outputs = model(data)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    
                    train_loss += loss.item()
                    _, predicted = outputs.max(1)
                    train_total += labels.size(0)
                    train_correct += predicted.eq(labels).sum().item()
                
                # Validation phase
                model.eval()
                val_loss = 0.0
                val_correct = 0
                val_total = 0
                
                with torch.no_grad():
                    for batch in val_loader:
                        if isinstance(batch['data'], dict):
                            data = {k: v.to(device) for k, v in batch['data'].items()}
                        else:
                            data = batch['data'].to(device)
                        
                        labels = batch['label'].to(device)
                        outputs = model(data)
                        loss = criterion(outputs, labels)
                        
                        val_loss += loss.item()
                        _, predicted = outputs.max(1)
                        val_total += labels.size(0)
                        val_correct += predicted.eq(labels).sum().item()
                
                # Calculate metrics
                train_acc = 100. * train_correct / train_total
                val_acc = 100. * val_correct / val_total
                avg_train_loss = train_loss / len(train_loader)
                avg_val_loss = val_loss / len(val_loader)
                
                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    # Save best model
                    torch.save(model.state_dict(), f"{self.config.models_dir}/{modality}_best.pth")
                
                # Log epoch results
                self.results_manager.log_epoch_results(
                    modality, epoch + 1, train_acc, val_acc, 
                    avg_train_loss, avg_val_loss, adapted_params["learning_rate"]
                )
                
                logger.info(f"  Epoch {epoch+1}/{adapted_params['epochs']}: "
                           f"Train {train_acc:.2f}%, Val {val_acc:.2f}%")
            
            # Log training end
            final_results = {
                "best_val_accuracy": best_val_acc,
                "final_train_accuracy": train_acc,
                "epochs_trained": adapted_params["epochs"]
            }
            self.results_manager.log_training_end(modality, final_results)

def run_adaptive_fl_experiment(time_budget_hours: float = 1.0):
    """Run the adaptive FL experiment with specified time budget"""
    print("🚀 Adaptive Multi-Modal Federated Learning Experiment")
    print("="*60)
    print(f"⏰ Time Budget: {time_budget_hours} hours")
    print("🧠 Features: Adaptive dataset discovery, intelligent scheduling, comprehensive results")
    print("="*60)
    
    try:
        # Initialize system
        system = AdaptiveMultiModalFL(time_budget_hours=time_budget_hours)
        
        # Run experiment
        results = system.run_adaptive_experiment()
        
        if results:
            print(f"\n🎉 Experiment completed successfully!")
            print(f"📊 Results saved to: {system.config.results_dir}")
            print(f"📈 Visualizations saved to: {system.config.plots_dir}")
            print(f"🤖 Models saved to: {system.config.models_dir}")
            
            # Print quick summary
            print(f"\n📋 Quick Summary:")
            for modality, result in results.items():
                acc = result.get("best_val_accuracy", 0)
                time = result.get("training_duration_minutes", 0)
                print(f"  {modality}: {acc:.2f}% accuracy in {time:.1f} minutes")
        
        return results
        
    except Exception as e:
        logger.error(f"Experiment failed: {e}")
        import traceback
        print(traceback.format_exc())
        return None

if __name__ == "__main__":
    # Run adaptive experiment with 1 hour time budget
    results = run_adaptive_fl_experiment(time_budget_hours=1.0)
    
    if results:
        print("\n✅ Adaptive Multi-Modal FL Experiment Complete!")
        print("📁 Check the results directory for comprehensive outputs!")
    else:
        print("\n❌ Experiment failed")

🚀 Adaptive Multi-Modal Federated Learning Experiment
⏰ Time Budget: 1.0 hours
🧠 Features: Adaptive dataset discovery, intelligent scheduling, comprehensive results

🎉 Experiment completed successfully!
📊 Results saved to: /kaggle/working/adaptive_results
📈 Visualizations saved to: /kaggle/working/plots
🤖 Models saved to: /kaggle/working/models

📋 Quick Summary:
  sensor: 27.42% accuracy in 0.1 minutes
  vision: 32.38% accuracy in 1.0 minutes

✅ Adaptive Multi-Modal FL Experiment Complete!
📁 Check the results directory for comprehensive outputs!
